In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Считываем данные из файла
df = pd.read_parquet("Задание/train_data.pqt")

# Выбираем метрики для модели
col = ['balance_amt_avg', 'channel_code', 'city_type', 'ogrn_month', 'ogrn_year', 'ft_registration_date', 'max_founderpres', 'okved', 'segment', 'sum_of_paym_2m', 'sum_of_paym_1y', 'sum_a_oper_3m', 'sum_c_oper_3m', 'sum_cred_e_oper_3m', 'sum_cred_g_oper_3m', 'sum_cred_h_oper_3m', 'start_cluster']
# col = ['start_cluster'] # Тестовая (чем меньше метрик, тем быстрее модель)

# Приводим данные к числовому виду
X = pd.DataFrame()
for i in col:
    if df[i].dtype.name != 'object':
        X[i]=df[i].copy()
        X.loc[X[i].isna(), i]=X[i].median()
    else:
        X[i]=pd.factorize(df[i])[0]

# Разделяем данные на признаки (X) и целевую переменную (y)
y = df['end_cluster']  # целевая переменная

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и Обучаем модель случайного леса
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#Считаем значимость метрик
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
ar_f.sort(reverse=True)
for i in ar_f:
    print(i)

# Предсказываем значения на тестовой выборке
y_pred = model.predict(X_test)

# Оцениваем качество модели
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




# Считываем данные из тестовой метрики
test_df = pd.read_parquet("Задание/test_data.pqt")

# Преобразовываем тестовую таблицу к численным значениям
X = pd.DataFrame()
for i in col:
    if test_df[i].dtype.name != 'object':
        X[i]=test_df[i].copy()
        X.loc[X[i].isna(), i]=X[i].median()
    else:
        X[i]=pd.factorize(test_df[i])[0]

# Предсказываем значения
model.predict(X)
test_probabilities = model.predict_proba(X)

test_probabilities

In [ ]:
test_df.id.unique()

In [ ]:
# Создание DataFrame с вероятностями перехода в каждый кластер для каждой строчки
cluster_probabilities_df = pd.DataFrame(test_probabilities, columns=model.classes_)

# Добавление целевой переменной к DataFrame с вероятностями
# cluster_probabilities_df['end_cluster'] = y_test.values
cluster_probabilities_df.index = test_df['id']
cluster_probabilities_df
# probabilities = model.predict_proba(X_test)

In [ ]:
cluster_probabilities_df.to_csv('results/sample_submission.csv', delimetr=',')